In [1]:
import numpy as np
from interest_similarity import find_similar_user_interest
from recommendation_function import get_recommended_items
from load_data import *
import warnings
warnings.filterwarnings("ignore")
# 노트북 1분 41초

In [2]:
# user 3은 video만 있음, place, product 없음
# user 20001은 place만 있음, product, video 없음
# user 26232는 product만 있음, place, video 없음 

# 신규 유저는 관심항목 작성
user_id = 3
new_user_item = [1,0,1]
new_user_outdoor = [0,0,0,0,0,0,0,0,0,1]
num_recommendations = 10
case2_dict = { 'place' : [case2_sgd_rating_place, place, case2_user_place, case2_user_place_idx, case2_user_latent_place, user_place],
         'video' : [case2_sgd_rating_video, video, case2_user_video, case2_user_video_idx, case2_user_latent_video, user_video],
         'product' : [case2_sgd_rating_product, product, case2_user_product, case2_user_product_idx, case2_user_latent_product, user_product]}
item = 'product'
item_list = list(case2_dict.keys())

# Case 2만 구현
# if : 기존 유저, else : 신규 유저 구분하기
if user_id in user['idx'].values:
    recomm_list = get_recommended_items(user_id, item, item_list, case2_dict, num_recommendations)
else:
    new_user_id = find_similar_user_interest(item, case2_dict, user_interest, new_user_item, new_user_outdoor)
    recomm_list = get_recommended_items(new_user_id, item, item_list, case2_dict, num_recommendations)
print(f"user {user_id}에게 추천해줄 {num_recommendations}개 {item} idx : {recomm_list}")

user 3는 product에 대한 행동내역이 없음
user 3는 video에 대한 행동이 존재함
video user latent에서 user 3과 유사한 user 찾기
10538가 product에 대한 행동이 없음
18752가 product에 대한 행동이 없음
14651가 product에 대한 행동이 없음
21260가 product에 대한 행동이 없음
7422가 product에 대한 행동이 없음
23862가 product에 대한 행동이 없음
16801가 product에 대한 행동이 없음
1060가 product에 대한 행동이 없음
3297가 product에 대한 행동이 없음
8006가 product에 대한 행동이 없음
12008가 product에 대한 행동이 없음
1503가 product에 대한 행동이 없음
14350가 product에 대한 행동이 없음
10480가 product에 대한 행동이 없음
14754가 product에 대한 행동이 없음
user 3과 가장 유사한 user : 10928, cos : 0.5263024773868127
user 3에게 추천해줄 10개 product idx : [4495, 5042, 6357, 4304, 5200, 5238, 3976, 3654, 1994, 542]


In [59]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import random

def item_latent_cos(user_id, original_item, item_list, case2_dict):
    exist_action = False
    item = ''
    for item_category in item_list:
        if user_id in case2_dict[item_category][3]['idx'].values:
            exist_action = True
            item = item_category
            print(f'user {user_id}는 {item_category}에 대한 행동이 존재함')
            break
    if exist_action is False:
        """ 모든 아이템에 대한 행동이 없는 유저에게 추천하는 함수 추가? """
        print(f'user {user_id}는 모든 아이템에 대한 행동이 없음')

    print(f'{item} user latent에서 user {user_id}과 유사한 user 찾기')
    drop_user_place_index = int(case2_dict[item][3][case2_dict[item][3]['idx'] == user_id].iloc[:,3]) # user_id에 대한 index 번호 추출
    all_item = case2_dict[item][2].loc[drop_user_place_index]
    non_zero_columns = all_item[all_item != 0].index.tolist()
    select_item = int(random.choice(non_zero_columns))
    cosine_sim_matrix = cosine_similarity(case2_item_latent_place) # user_latent에 대한 코사인 유사도 계산
    cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=case2_item_latent_place.index, columns=case2_item_latent_place.index)
    user_similarities = cosine_sim_df.loc[select_item] # user_id의 코사인 유사도 값을 가져옴
    user_similarities[select_item] = -1 # user_id를 선택하지 않도록 -1을 해줌
    sorted_user_similarities = user_similarities.sort_values(ascending=False) # 유사도가 높은 순으로 정렬
    
    top_recommendations = sorted_user_similarities.index.tolist()[:num_recommendations] # 상위 N개만큼 뽑아옴
    print(top_recommendations)
    recommendations_result = case2_dict[item][1].iloc[top_recommendations]['idx'].tolist() # 아이템 idx 매핑
    
    print(f'user {user_id}과 사용했던 아이템 {select_item} 와 유사도가 높은 idx: {recommendations_result}')
    return new_user_id

In [60]:
user_id = 3
item = 'product'
item_latent_cos(user_id, item, item_list, case2_dict)

user 3는 video에 대한 행동이 존재함
video user latent에서 user 3과 유사한 user 찾기
2145    0.555699
1054    0.514114
2091    0.512856
3270    0.502108
777     0.496259
          ...   
2852   -0.471265
110    -0.486531
3105   -0.511193
365    -0.520438
94     -1.000000
Name: 94, Length: 4697, dtype: float64
[2145, 1054, 2091, 3270, 777, 4690, 3202, 1359, 2469, 4421]


IndexError: positional indexers are out-of-bounds